# Control flow in Bash: `if`, `while` and `trap`s

```{objectives}
1. Be able to perform simple tests and influence execution
2. Keep control even when your script is sent a signal
```

```{admonition} To follow along

Please navigate to the `example/control_flow` directory.
```

In [1]:
cd ../../examples/control_flow

## Control flow statements

The most important control flow statements in Bash are 
- `if`/`else` blocks
- `while` loops

They are used in conjunction with anything that has a *return status*.

In particular:
- the `test` command,  equivalent to `[ ]`, or with the command `[[ ]]` (a Bash extension).
- Arithmetic comparisons using `(( <condition> ))`

But they can be used with any process that terminates.
A return status equal to `0` means "True", and all other values mean "False", a `!` negates the value of an expression. 

```{exercise} Conditional expressions

Search the Bash manual for the section on `CONDITIONAL EXPRESSIONS`.

1. What expression can be used to test if a file or directory already exists?
2. How to compare two numbers?

```{solution}
1. `[ -a a/path/ ]` will check if `a/path/` exists
2. `[ $N1 -lt $N2 ]` will check if $N1 is **l**ess **t**han $N2. There are 5 other related checks. `ARIHTMETIC EVALUATION` can also be used in this case: `(( N1 < N2 ))`
```

```{tip}
Control expressions have `exit` statuses
```

Note: Spaces around the square brackets are important!


## `if`-`else` statements

We can check if a file exists, and if not, we can create it:

In [2]:
if ! [ -f myfile.txt ]
then
   touch myfile.txt
   echo "file created now"
else
   echo "file exists already"
fi


file exists already


## `&&`, `||` and `!`

`&&`, `||` and `!` are boolean logic operators that can be used to combine commands in a *list*, while at the same time combining their return codes.

Interestingly:
- A list of commands joined by `&&` will run only until one of them fails.
- A list of commands joined by `||` will run only until one of them succeeds.

If the body of the `if` statements are short, `&&` and `||` can be used more conveniently to simulate an `if` statement (but perhaps in a less readable way):

In [3]:
! [ -f myfile.txt ] && { touch myfile.txt; echo "file created now"; } || echo "file exists already"

file exists already


In this case, we have collected some steps together in braces `{}`.

`````{exercise} Be precise!
Actually, 
````bash
condition && then_action || else_action
````
is not a precise translation of 
````bash
if condition
then then_action
else else_action
fi
````

Can you figure out the case where the two expressions behave differently? How can you fix the first expression so that it behaves more precisely like the second one?

````{solution}
When the `condition` evaluates to `true`, but the `then_action` evaluates to `false`, then the `else_action` will be executed anyway.

This more complex expression behaves more precisely like an `if` statement:
```bash
condition && { then_action || true ; } || else_action
```
````
`````

## While loops

The syntax is similar to for loops, but the header is different:

In [4]:
while ! [ -f myfile.txt ]
do
   echo "File does not yet exist"
   # waiting a little before repeating the loop
   sleep 5
done

`````{exercise} Until loops
The loop above could be rewritten more conveniently using a `until` loop.
What should be changed?

   ````{solution}

      ```
      until [ -f myfile.txt ]
      do
          echo "File does not yet exist"
          # waiting a little 
          sleep 2
      done
      ```

   ````   
     
`````

## Dealing with signals: traps

When your script receives a signal that it does know how to handle,
by default it terminates.

With the `trap` command, it is possible to catch and handle some of the signals,
typically before aborting, for example cleaning up:


In [5]:
cleanup(){
  echo "Cleanup before interrupt and exit"
  exit 0
}

(
trap "cleanup" SIGINT

while true
do
  echo "sleeping..."
  sleep 1
done
) & 



sleep 5
kill -SIGINT %1 # Sending the signal to the process in the background.

[1] 40521
sleeping...
sleeping...
sleeping...
sleeping...
sleeping...
sleeping...
